<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: spec2, extract_2d step

<span style="color:red"> **Instruments Affected**</span>: NIRSpec 

Tested on CV3 data

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Testing Data Set](#data_ID) <br> [Run the JWST pipeline and assign_wcs validation tests](#pipeline_ID): [FS Full-Frame test](#FULLFRAME), [FS ALLSLITS test](#ALLSLITS), [MOS test](#MOS) <br> [About This Notebook](#about_ID)<br> [Results](#results) <br>

</div>

<a id="imports_ID"></a>
# Imports
The library imports relevant to this notebook are aready taken care of by importing PTT.

* astropy.io for opening fits files
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

NOTE: This notebook assumes that the pipeline version to be tested is already installed and its environment is activated.

To be able to run this notebook you need to install nptt. 

If all goes well you will be able to import PTT.

[Top of Page](#title_ID)

In [ ]:
import os
from astropy.io import fits

import jwst
from jwst.extract_2d import Extract2dStep

# The latest version of NPTT is installed in the requirements text file at:
# /jwst_validation_notebooks/requirements.txt

# import NPTT
import nirspec_pipe_testing_tool as nptt

In [ ]:
# VERSIONS

# print the jwst pipeline version
print("Using JWST pipeline version: ", jwst.__version__)

# Make sure that the version used is the right one

# to get the latest version from 'master' run:
# subprocess.run(['pip', 'install', 'git+https://github.com/spacetelescope/nirspec_pipe_testing_tool@master'])

latest_nptt_stable_version = '1.1.1'
nptt_version = nptt.__version__

if nptt_version != latest_nptt_stable_version:
    print("\n* WARNING - The code might break because the version used is not the latest stable version:")
    print("            Latest stable version is ", latest_nptt_stable_version)
    print("            Using NPTT version ", nptt_version)

<a id="intro_ID"></a>
# Introduction

We compared Institute's pipeline product of the assign_wcs step with our benchmark files, or with the intermediary products from the ESA pipeline, which is completely independent from the Institute's. The comparison file is referred to as 'truth'. We calculated the relative difference and expected it to be equal to or less than computer precision:  relative_difference = absolute_value( (Truth - ST)/Truth )  <= 1x10^-7. 

For the test to be considered PASSED, every single slit (for FS data), slitlet (for MOS data) or slice (for IFU data) in the input file has to pass. If there is any failure, the whole test will be considered as FAILED. 

The code for this test can be obtained at: https://github.com/spacetelescope/nirspec_pipe_testing_tool/blob/master/nirspec_pipe_testing_tool/calwebb_spec2_pytests/auxiliary_code/check_corners_extract2d.py. Multi Object Spectroscopy (MOS), the code is in the same repository but is named ```compare_wcs_mos.py```, and for Integral Field Unit (IFU) data, the test is named ```compare_wcs_ifu.py```.
The input file is defined in the variable ```input_file``` (see section [Testing Data Set and Variable Setup](#data_ID)).

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/extract_2d/main.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/extract_2d


### Calibration WG Requested Algorithm: 

A short description and link to the page: 
https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Path-Loss+Correction


### Defining Term
Acronymns used un this notebook:

pipeline: calibration pipeline

spec2: spectroscopic calibration pipeline level 2b

PTT: NIRSpec pipeline testing tool (https://github.com/spacetelescope/nirspec_pipe_testing_tool)


[Top of Page](#title_ID)

<a id="data_ID"></a>
# Testing Data Set

All testing data is from the CV3 campaign. We chose these files because this is our most complete data set, i.e. all modes and filter-grating combinations.

Data used was for testing was only FS and MOS, since extract_2d is skipped for IFU. Data sets are:
- FS_FULLFRAME_G395H_F290LP
- FS_ALLSLITS_G140H_F100LP 
- MOS_G140M_LINE1 

The testing data can be located at ```path_to_nirspec_vault/prelaunch_data/testing_sets/b7.1_pipeline_testing/test_data_suite```

In that directory you will find a subdirectory for CV3 data for FS, MOS, and IFU, as well as other campaigns. This notebook is specifically written for CV3 data.



[Top of Page](#title_ID)

Set the following variables to the file you are interested in testing:

- output_directory = path where you want intermediary files and plots to be saved in

- input_file = this is the count rate file, e.g. jwtest1_NRS1_rate.fits

- mode_used = supported modes are: FS, MOS, IFU, BOTS, MOS_sim, dark, image, confirm, taconfirm, wata, msata, focus, mimf

- raw_data_root_file = CV3 raw data file


<a id="pipeline_ID"></a>
# Run the JWST pipeline and extract_2d validation tests

The pipeline can be run from the command line in two variants: full or per step.

Tu run the spec2 pipeline in full use the command: 

$ strun jwst.pipeline.Spec2Pipeline jwtest_rate.fits

Tu only run the extract_2d step, use the command:

$ strun jwst.extract_2d.Extract2dStep jwtest_previous_step_output.fits


These options are also callable from a script with the testing environment active. The Python call for running the pipeline in full or by step are:

$\gt$ from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

$\gt$ Spec2Pipeline.call(jwtest_rate.fits)
 
or
 
$\gt$ from jwst.extract_2d import Extract2dStep
 
$\gt$ Extract2dStep.call(jwtest_previous_step_output.fits)


PTT can run the spec2 pipeline either in full or per step, as well as the imaging pipeline in full. In this notebook we will use PTT to run the pipeline and the validation tests. To run PTT, follow the directions in the corresponding repo page.

[Top of Page](#title_ID)

-> For each mode, the following variables will need to be set:

- output_directory = path where you want intermediary files and plots to be saved in

- input_file = this is the count rate file, e.g. jwtest1_NRS1_rate.fits

- mode_used = supported modes are: FS, MOS, IFU, BOTS, MOS_sim, dark, image, confirm, taconfirm, wata, msata, focus, mimf

- raw_data_root_file = CV3 raw data file


<a id="Common_variables"></a>
# Common Variables

In [ ]:
# Set common variables for entire notebook

# paths
vault_path = os.path.join(os.environ.get('WIT4_PATH'), 'nirspec_vault')
esa_files_path = os.path.join(vault_path, 'prelaunch_data/testing_sets/b7.1_pipeline_testing/test_data_suite/')
test_data_path = '/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data'

# accepted threshold difference with respect to benchmark files
extract_2d_threshold_diff = 4

# define benchmark (or 'truth') file
compare_assign_wcs_and_extract_2d_with_esa = False


<a id="FULLFRAME"></a>
# FS FULL-FRAME test

In [ ]:
# Set the variables for Fixed Slit (FS) data
data_directory = os.path.join(test_data_path, 'FS_FULL_FRAME/G395H_opaque')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_assign_wcs.fits')
mode_used = 'FS'

In [ ]:
# ESA intermediary products
raw_data_root_file = 'NRSSMOS-MOD-G3H-02-5344120942_1_491_SE_2015-12-10T12h18m25.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [ ]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

In [ ]:
# Run the validation test
%matplotlib inline
result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_FSwindowcorners(pipe_result, 
                                                              truth_file=truth_file,
                                                              esa_files_path=esa_files_path,
                                                              extract_2d_threshold_diff=extract_2d_threshold_diff)

In [ ]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation for FS FULL-FRAME test passed? ", result)


<a id="ALLSLITS"></a>
# FS ALLSLITS test

In [ ]:
# Set the variables for Fixed Slit (FS) data
data_directory = os.path.join(test_data_path, 'FS_ALLSLITS/G140H_F100LP')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_F100LP_assign_wcs.fits')
mode_used = 'FS'

In [ ]:
# ESA intermediary products
raw_data_root_file = 'NRSSDRK-ALLSLITS-5345150216_1_491_SE_2015-12-11T15h40m25.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_F100LP_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [ ]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

In [ ]:
# Run the validation test
%matplotlib inline
result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_FSwindowcorners(pipe_result, 
                                                              truth_file=truth_file,
                                                              esa_files_path=esa_files_path,
                                                              extract_2d_threshold_diff=extract_2d_threshold_diff)

In [ ]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation test for FS ALLSLITS passed? ", result)


<a id="MOS"></a>
# MOS test

In [ ]:
# Set the variables 
data_directory = os.path.join(test_data_path, 'MOS/G140M_LINE1')
input_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_msa_flagging.fits')
mode_used = 'MOS'
msa_conf_name = os.path.join(data_directory,'V8460001000101_msa.fits')

In [ ]:
# ESA intermediary products
raw_data_root_file = 'NRSV84600010001P0000000002101_4_491_SE_2016-01-17T17h34m08.fits'
esa_files_full_path = "".join([esa_files_path, mode_used, "_CV3_cutouts/ESA_Int_products"])

# pipeline intermediary products
truth_file = os.path.join(data_directory, 'final_output_caldet1_NRS1_extract_2d_truth.fits')

if not compare_assign_wcs_and_extract_2d_with_esa:
    esa_files_path, raw_data_root_file = None, None
else:
    truth_file = None

In [ ]:
# Run the pipeline step
step_output_file = input_file.replace(".fits", "_extract_2d.fits")
pipe_result = Extract2dStep.call(input_file)
#pipe_result.save(step_output_file)   # this is to save the pipeline output to a file

In [ ]:
# Run the validation test
%matplotlib inline
mos_result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_MOSwindowcorners(pipe_result,
                                                               msa_conf_name,
                                                               truth_file=truth_file,
                                                               esa_files_path=esa_files_path,
                                                               extract_2d_threshold_diff= extract_2d_threshold_diff)

In [ ]:
# Quickly see if the test passed 
   
print("Did assign_wcs validation for MOS test passed? ", mos_result)


<a id="results"></a>
## Results

If the test **PASSED** this means that all slits, slitlets, or slices individually passed the test. However, if ony one individual slit (for FS data), slitlet (for MOS data) or slice (for IFU data) test failed, the whole test will be reported as **FAILED**.

<a id="about_ID"></a>
## About this Notebook
**Author:** Maria A. Pena-Guerrero, Staff Scientist II - Systems Science Support, NIRSpec
<br>**Updated On:** 04/09/2020

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 